# SPU over local network

This example spins up a 2-party SPU network. Both parties listen on 0.0.0.0, and will connect to each other over the local network instead of through the loopback interface (localhost).


First, start an example 2-party SecretFlow network in simulation mode:


In [ ]:
import secretflow
from secretnote.compat.secretflow.device.driver import SFConfigSimulationFullyManaged

secretflow.shutdown()

secretflow_config = SFConfigSimulationFullyManaged(parties=["alice", "bob"])
secretflow.init(**secretflow_config.dict())

alice = secretflow.PYU("alice")
bob = secretflow.PYU("bob")

Figure out what the local IP address is for this machine. Depending on your network configuration, there may be several applicable addresses (e.g. one on Wireless and one on Ethernet). We will be using [`netifaces`][netifaces] to find the first interface that is up and has an `AF_INET` address assigned to it:

[netifaces]: https://pypi.org/project/netifaces/


In [ ]:
import netifaces

In [ ]:
for iface in netifaces.interfaces():
    if iface.startswith("lo"):
        # skip loopback interface
        continue
    if iface.startswith("vbox"):
        # skip virtualbox interfaces
        continue
    details = netifaces.ifaddresses(iface)
    if netifaces.AF_INET not in details:
        # skip interfaces without an IP address
        continue
    inet_addr = details[netifaces.AF_INET][0]["addr"]
    break
else:
    raise RuntimeError("No suitable network interface found")

print(f"Using interface {iface} with address {inet_addr}")

Find two ports that are available on this machine. We will be using [`portpicker`][portpicker]:

[portpicker]: https://pypi.org/project/portpicker/


In [ ]:
import portpicker

In [ ]:
port_for_alice = portpicker.pick_unused_port()
port_for_bob = portpicker.pick_unused_port()

alice_bind_to = f"127.0.0.1:{port_for_alice}"
bob_bind_to = f"127.0.0.1:{port_for_bob}"

connect_to_alice_via = f"{inet_addr}:{port_for_alice}"
connect_to_bob_via = f"{inet_addr}:{port_for_bob}"

print(
    f"""Network layout:
alice: {alice_bind_to} -> {connect_to_alice_via}
bob:   {bob_bind_to} -> {connect_to_bob_via}"""
)

Spin up the SPU network:


In [ ]:
from secretnote.compat.spu import (
    SPUClusterDef,
    SPUConfig,
    SPURuntimeConfig,
    SPUProtocolKind,
    SPUFieldType,
    SPUNode,
)

mpc_config = SPUConfig(
    cluster_def=SPUClusterDef(
        nodes=[
            SPUNode(
                party="alice",
                listen_addr=alice_bind_to,
                address=connect_to_alice_via,
            ),
            SPUNode(
                party="bob",
                listen_addr=bob_bind_to,
                address=connect_to_bob_via,
            ),
        ],
        runtime_config=SPURuntimeConfig(
            protocol=SPUProtocolKind.SEMI2K,
            field=SPUFieldType.FM128,
        ),
    ),
)

mpc = secretflow.SPU(**mpc_config.dict())
mpc.init()

Run an example JAX program on the SPU network:


In [ ]:
import jax.numpy as jnp

In [ ]:
def dot(x, y):
    return jnp.dot(x, y)

In [ ]:
product = mpc(dot)(
    secretflow.to(alice, jnp.arange(10)).to(mpc),
    secretflow.to(bob, jnp.arange(10, 20)).to(mpc),
)
secretflow.reveal(product)

Stop everything:


In [ ]:
mpc.shutdown()
secretflow.shutdown()